In [1]:
import os

os.environ['R_HOME'] = '/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/R'

In [2]:
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint
import scanpy as sc
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

Setting environment seed: 42


/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


In [3]:
sc_ad = sc.read_h5ad('/home/comp/cszrwang/data/MOB/mob_sc_celltype.h5ad')
st_ad = sc.read_h5ad('/home/comp/cszrwang/data/MOB/mob_st.h5ad')

sc.pp.filter_genes(st_ad,min_cells=1)
sc.pp.filter_genes(sc_ad,min_cells=1)
sc.pp.filter_cells(st_ad,min_genes=1)
sc.pp.filter_cells(sc_ad,min_genes=1)

In [4]:
sc_ad.obs['celltype']

WT1_AAACCTGAGCTGCGAA      GC
WT1_AAACCTGGTTTGGCGC      GC
WT1_AAACGGGAGATCCGAG      GC
WT1_AAACGGGGTAAGTGGC      GC
WT1_AAAGATGTCATCGGAT     PGC
                        ... 
OC2_TAAGTGCGTTTGTTGG    M/TC
OC2_TACGGTAGTAAATGTG    M/TC
OC2_TACTTACGTAGGCATG    M/TC
OC2_TAGTTGGTCGCGATCG    M/TC
OC2_TTGCGTCAGGATCGCA    M/TC
Name: celltype, Length: 12801, dtype: category
Categories (5, object): ['EPL-IN', 'GC', 'M/TC', 'OSNs', 'PGC']

In [5]:
spoint_model = Spoint.init_model(sc_ad,st_ad,celltype_key='celltype',deg_method='t-test',sm_size=100000,use_gpu=True)

Setting global seed: 42
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
### Finding marker genes...
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)
celltype
EPL-IN    200
GC        200
M/TC      200
PGC       200
OSNs      178
Name: count, dtype: int64
filtered out 2 cells that have less than 1 genes expressed
### Used gene numbers: 909
### Initializing sample probability
### Genetating simulated spatial data using scRNA data with mode: unbalance
### Genetating simulated spatial data using scRNA data with mode: sqrt
### Genetating simulated spatial data using scRNA data with mode: balance


In [6]:
spoint_model.train(max_steps=5000, batch_size=1024)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 1/100:   0%|                                                                                                                              | 0/100 [00:00<?, ?it/s]

/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/scvi/distributions/_negative_binomial.py:476: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(
/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/scvi/distributions/_negative_binomial.py:476: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(


Epoch 2/100:   1%|▉                                                                                             | 1/100 [00:01<02:16,  1.38s/it, loss=1.65e+03, v_num=1]

/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/scvi/distributions/_negative_binomial.py:476: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:10<00:00,  1.51it/s, loss=1.28e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:10<00:00,  1.41it/s, loss=1.28e+03, v_num=1]


Step 5000: Test inference loss=-0.950: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [21:23<00:00,  3.90it/s]


In [7]:
pre = spoint_model.deconv_spatial()

In [8]:
pre

,GC,PGC,OSNs,M/TC,EPL-IN
17x17,0.790429,0.000000,0.000000,0.174712,0.027894
18x17,0.121071,0.222687,0.033223,0.406782,0.216237
20x17,0.165138,0.056000,0.248388,0.357678,0.172796
19x17,0.177339,0.055705,0.258092,0.297724,0.211141
22x17,0.337095,0.000000,0.000000,0.639230,0.017492
...,...,...,...,...,...
19x27,0.229130,0.215519,0.014663,0.304568,0.236120
28x27,0.163427,0.360567,0.000000,0.288859,0.185020
24x29,0.206729,0.278373,0.000000,0.315397,0.199299
25x29,0.182138,0.313323,0.000000,0.298607,0.205827


In [9]:
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/MOB/SPACEL/"

In [12]:
pre.to_csv(output_file_path + '/SPACEL_result.csv')

In [13]:
st_ad_show = spoint_model.st_ad

In [14]:
st_ad_show

AnnData object with n_obs × n_vars = 280 × 909
    obs: 'index', 'x', 'y', 'n_genes', 'batch', 'GC', 'PGC', 'OSNs', 'M/TC', 'EPL-IN'
    var: 'n_cells'
    uns: 'celltypes'
    layers: 'norm'